In [1]:
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec

/home/sunild/anaconda3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# read ner dataset
with open("data/hindi_ner_dataset/wikiann-hi.bio",'r') as file:
    lines=file.readlines()
    print(len(lines))

81264


In [4]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModel.from_pretrained('ai4bharat/indic-bert')


Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
len(tokenizer.get_vocab().keys())
#  Returns the added tokens in the vocabulary as a dictionary of token to index.


200000

In [32]:
dict_vocab=tokenizer.get_vocab()
dict_vocab['थल']

44741

In [49]:
tokenizer.convert_ids_to_tokens(0),tokenizer.convert_ids_to_tokens(1),tokenizer.convert_ids_to_tokens(2),tokenizer.convert_ids_to_tokens(3),tokenizer.convert_ids_to_tokens(4)

('<pad>', '<unk>', '[CLS]', '[SEP]', '[MASK]')

In [33]:
tokenizer.convert_tokens_to_ids('थल')

44741

In [78]:
a=tokenizer('थल')


In [80]:
a

{'input_ids': [2, 78045, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [79]:

model.forward(a.input_ids)

AttributeError: 'list' object has no attribute 'size'

In [73]:
help(model)

Help on AlbertModel in module transformers.models.albert.modeling_albert object:

class AlbertModel(AlbertPreTrainedModel)
 |  AlbertModel(config, add_pooling_layer=True)
 |  
 |  The bare ALBERT Model transformer outputting raw hidden-states without any specific head on top.
 |  
 |  This model inherits from [`PreTrainedModel`]. Check the superclass documentation for the generic methods the
 |  library implements for all its model (such as downloading or saving, resizing the input embeddings, pruning heads
 |  etc.)
 |  
 |  This model is also a PyTorch [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) subclass.
 |  Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage
 |  and behavior.
 |  
 |  Args:
 |      config ([`AlbertConfig`]): Model configuration class with all the parameters of the model.
 |          Initializing with a config file does not load the weights associated with the model, only th

In [68]:
tokenizer.model_input_names

['input_ids', 'token_type_ids', 'attention_mask']

In [55]:
tokenizer.convert_tokens_to_string(['का','थल','राम','कथ','जम','कॆ'])

'काथलरामकथजमकॆ'

In [65]:
tokenizer.special_tokens_map

{'bos_token': '[CLS]',
 'eos_token': '[SEP]',
 'unk_token': '<unk>',
 'sep_token': '[SEP]',
 'pad_token': '<pad>',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [59]:
tokenizer.tokenize('जयपुुुर सालासर चुरु सिकर नागॊर कॆ')

['▁जय', 'पर', '▁सल', 'सर', '▁चर', '▁', 'सकर', '▁नगर', '▁क']

In [58]:
tokenizer.num_special_tokens_to_add()

2

In [56]:
help(tokenizer)

Help on AlbertTokenizerFast in module transformers.models.albert.tokenization_albert_fast object:

class AlbertTokenizerFast(transformers.tokenization_utils_fast.PreTrainedTokenizerFast)
 |  AlbertTokenizerFast(vocab_file=None, tokenizer_file=None, do_lower_case=True, remove_space=True, keep_accents=False, bos_token='[CLS]', eos_token='[SEP]', unk_token='<unk>', sep_token='[SEP]', pad_token='<pad>', cls_token='[CLS]', mask_token='[MASK]', **kwargs)
 |  
 |  Construct a "fast" ALBERT tokenizer (backed by HuggingFace's *tokenizers* library). Based on
 |  [Unigram](https://huggingface.co/docs/tokenizers/python/latest/components.html?highlight=unigram#models). This
 |  tokenizer inherits from [`PreTrainedTokenizerFast`] which contains most of the main methods. Users should refer to
 |  this superclass for more information regarding those methods
 |  
 |  Args:
 |      vocab_file (`str`):
 |          [SentencePiece](https://github.com/google/sentencepiece) file (generally has a *.spm* exten

In [5]:
tokenizer('टोमाकोमाई')

{'input_ids': [2, 166973, 29353, 1038, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [11]:
tokenizer('हिमाचल_प्रदेश')

{'input_ids': [2, 524, 50708, 5513, 6219, 156429, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

~T_destination

In [4]:
def forward():
    pass